In [1]:
#Check Python Version
import shutil
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [2]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


2023-02-17 16:30:18.148028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 16:30:18.613508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 16:30:18.613617: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 16:30:20.477389: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [5]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import apache_beam as beam

from absl import logging

# from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
# from base_example_gen_executor_v2.py import BaseExampleGenExecutorV2
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Local files I am exploring
from component_v2 import CopyExampleGen
from base_example_gen_executor_v2 import BaseExampleGenExecutorV2

# Set up logging.
tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)
pp = pprint.PrettyPrinter()

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

TensorFlow version: 2.11.0
TFX version: 1.12.0


In [6]:
@beam.ptransform_fn
@beam.typehints.with_input_types(beam.Pipeline)
@beam.typehints.with_output_types(tf.train.Example)
def _TFDatasetToExample(  # pylint: disable=invalid-name
    pipeline: beam.Pipeline,
    exec_properties: Dict[str, Any],
    split_pattern: str
    ) -> beam.pvalue.PCollection:
    """Read a TensorFlow Dataset and create tf.Examples"""
    custom_config = json.loads(exec_properties['custom_config'])
    dataset_name = custom_config['dataset']
    # dataset_name = custom_config['dataset']
    split_name = custom_config['split']
    # split_name = custom_config['split']

    builder = tfds.builder(dataset_name)
    builder.download_and_prepare()

    return (pipeline
            | 'MakeExamples' >> tfds.beam.ReadFromTFDS(builder, split=split_name)
            | 'AsNumpy' >> beam.Map(tfds.as_numpy)
            | 'ToDict' >> beam.Map(dict)
            | 'ToTFExample' >> beam.Map(utils.dict_to_example)
            )

class TFDSExecutor(BaseExampleGenExecutorV2):
  def GetInputSourceToExamplePTransform(self) -> beam.PTransform:
    """Returns PTransform for TF Dataset to TF examples."""
    return _TFDatasetToExample

In [9]:
context = InteractiveContext()

In [12]:
# Tfrecord data
copy_example_gen = CopyExampleGen(
    input_base='./external_tfrecords/tfrecords/Split_train/',
    custom_config=
    {
        'dataset':'movielens/100k-ratings',
        'split':'train'
    },
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor)
    )
context.run(copy_example_gen)


INFO:absl:Running driver for CopyExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CopyExampleGen
INFO:absl:Generating examples.
INFO:absl:Load dataset info from /usr/local/google/home/alexanderho/tensorflow_datasets/movielens/100k-ratings/0.1.1
INFO:absl:Reusing dataset movielens (/usr/local/google/home/alexanderho/tensorflow_datasets/movielens/100k-ratings/0.1.1)
INFO:absl:Load dataset info from /usr/local/google/home/alexanderho/tensorflow_datasets/movielens/100k-ratings/0.1.1
INFO:absl:Load dataset info from /usr/local/google/home/alexanderho/tensorflow_datasets/movielens/100k-ratings/0.1.1
INFO:absl:Load dataset info from /usr/local/google/home/alexanderho/tensorflow_datasets/movielens/100k-ratings/0.1.1
INFO:absl:Load dataset info from /usr/local/google/home/alexanderho/tensorflow_datasets/movielens/100k-ratings/0.1.1
INFO:absl:Load dataset in

ExecutionResult(
    component_id: CopyExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CopyExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [ ]:
movies_stats_gen = tfx.components.StatisticsGen(
    examples=copy_example_gen.outputs['examples'])
context.run(movies_stats_gen, enable_cache=True)